# **1. Kết nối với google drive**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/BTL3- BERT and GPT/')
os.getcwd()

Mounted at /content/drive


'/content/drive/.shortcut-targets-by-id/1bs2wermM-w1hKEOcHyv-UmLHY5iag5cm/NLP 4+1/BTL3- BERT and GPT'

# **2. Install và import các thư viện cần thiết**

In [ ]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.1 MB/s eta 0:00:00


In [ ]:
import torch
import time
import pandas as pd
from tqdm.notebook import tqdm

from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from transformers import GPT2LMHeadModel, GPT2Model
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
from transformers import TextDataset, GPT2Tokenizer, AutoTokenizer

# **3. Set các biến cục bộ**

In [ ]:
import pickle

# saving
def save_file(data, path):
    with open(path + '.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
EPOCHS = 100
BATCH_SIZE = 32
path_data = './dataset/'                # đường dẫn data
path_pre_trained = './pre-trained/GPT/'  # đường dẫn chứa các model đã train

In [ ]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                              "bos_token": "<s>",
                              "eos_token": "</s>"})
tokenizer.add_tokens(["<bot>:"])

# save
save_file(tokenizer, path_pre_trained + 'tokenizer')

In [ ]:
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# **4. Xây dựng class để tạo dataset cho quá trình huấn luyện**

In [ ]:
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = pd.read_csv(path, usecols = [0,1])

        self.X = []
        for q,a in self.data.values:
            self.X.append('<s> '+ q +" <bot>: "+ a +' </s>')
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, max_length=96, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
chatData = ChatData(path_data + 'train.csv', tokenizer)
chatData =  DataLoader(chatData, batch_size=BATCH_SIZE)

<s> what canbe the cause for chlamydia not going away after treatment and no sexual intercourse after being diagnosed <bot>: i do not know how the chlamydia was treated was the dose appropriate and taken long enough were you compliant in taking it properly are you male or female do have more than one partner as much as i would like to address your question i really do not have enough information </s>


# **5. Xây dựng hàm để traning và trả lời câu hỏi từ model**

In [ ]:
def train(chatData, model, optim, epochs):
    loss_history = []
    for i in range(epochs):
        total_loss = 0
        print('======== Epoch {:} / {:} ========'.format(i + 1, epochs))
        for X, a in tqdm(chatData):
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            total_loss += loss.item()
            loss.backward()
            optim.step()
        avg_train_loss = total_loss / len(chatData)  
        loss_history.append(avg_train_loss)
        torch.save(model.state_dict(), path_pre_trained + "model_GPT.pt")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print(bot_give_answer("do i need to exercise to benefit from vitamin c"))
    return loss_history
    
def bot_give_answer(inp):
    inp = '<s> '+ inp +" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    mask = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=mask, max_length=96, pad_token_id=0, eos_token_id=50256)
    output = tokenizer.decode(output[0])
    return output

In [ ]:
# training
model.train()
optim = Adam(model.parameters(), lr=1e-3)

t0 = time.time()
loss_history = train(chatData, model, optim, epochs=EPOCHS)
time_train = time.time() - t0
# no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own

======== Epoch 1 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

  Average training loss: 4.98
<s> do i need to exercise to benefit from vitamin c <bot>: it is a vitamin that is a vitamin that is not a vitamin that is used for the treatment of vitamin c <bot>: a and the treatment of vitamin c12 and the treatment of vitamin c12 is not a vitamin that is used for the treatment of vitamin c12 and the treatment of vitamin c </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 2 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

  Average training loss: 2.32
<s> do i need to exercise to benefit from vitamin c <bot>: it is important that you exercise regularly and not just because you are not taking vitamin c you should not exercise or have a vitamin c to help your body fight off vitamin c and vitamin c </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 3 / 100 ========


...

  Average training loss: 0.12
<s> do i need to exercise to benefit from vitamin c <bot>: no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certainly not exercise is not required to increase absorption of vitamin c </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 99 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

  Average training loss: 0.12
<s> do i need to exercise to benefit from vitamin c <bot>: no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certainly not exercise is not required to increase absorption of vitamin c </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
======== Epoch 100 / 100 ========


  0%|          | 0/196 [00:00<?, ?it/s]

  Average training loss: 0.12
<s> do i need to exercise to benefit from vitamin c <bot>: no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certainly not exercise is not required to increase absorption of vitamin c </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [ ]:
val_compare = {
    'loss_history': loss_history,
    'time_train': time_train
}

# save
save_file(val_compare, path_pre_trained + 'val_compare')